In [2]:
import cv2
import easyocr
import numpy as np
from ultralytics import YOLO
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array

In [3]:
# Load YOLO model for license plate detection
yolo_model = YOLO("D:/anpr2/runs/detect/train_final.pt")  # Path to your YOLO model

In [4]:
# Initialize EasyOCR reader for text recognition (OCR)
ocr_reader = easyocr.Reader(['en'], gpu=True)

KeyboardInterrupt: 

In [ ]:
# Load the trained character recognition OCR model
ocr_model = tf.keras.models.load_model("D:/anpr2/ocr_model_final.keras")  # Load the OCR model

c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
def preprocess_for_ocr(cropped_image):
    gray = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)  # Thresholding
    denoised = cv2.GaussianBlur(thresh, (5, 5), 0)  # Denoising
    return denoised

In [ ]:
# Start capturing video from webcam
cap = cv2.VideoCapture(0)  # Use 0 for webcam, or provide a video file path

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Perform YOLO object detection
    results = yolo_model(frame)  # Perform object detection using YOLO

    # Extract bounding boxes and confidence scores from results
    pred = results[0].boxes  # Get the box predictions (x, y, width, height, confidence)
    
    # Loop through the detections
    for box in pred:
        confidence = box.conf.item()  # Get the confidence score
        if confidence > 0.5:  # Only consider detections with confidence > 50%
            x1, y1, x2, y2 = map(int, box.xywh[0])  # Extract bounding box coordinates
            license_plate_image = frame[y1:y2, x1:x2]  # Extract the license plate image
            
            # Check if the cropped image is valid (non-empty)
            if license_plate_image.size == 0:
                continue  # Skip this detection if the image is empty

            # Preprocess the license plate image for OCR model (resize to 28x28 and convert to grayscale)
            license_plate_image = cv2.cvtColor(license_plate_image, cv2.COLOR_BGR2GRAY)
            license_plate_image = cv2.resize(license_plate_image, (28, 28))
            license_plate_image = img_to_array(license_plate_image)
            license_plate_image = np.expand_dims(license_plate_image, axis=0)
            license_plate_image = license_plate_image / 255.0  # Normalize the image

            # Use OCR model to predict characters
            ocr_result = ocr_model.predict(license_plate_image)

            # Decode the predicted character(s) from the OCR model output
            predicted_class = np.argmax(ocr_result, axis=-1)

            # Ensure that predicted_class is a scalar value
            predicted_class = predicted_class.item()  # Convert to scalar if it's an array

            # Convert to character (assuming your OCR model recognizes A-Z and 0-9)
            if predicted_class < 26:
                predicted_char = chr(predicted_class + 65)  # A-Z
            else:
                predicted_char = str(predicted_class - 26)  # 0-9

            # Display the recognized text on the image
            cv2.putText(frame, f"License Plate: {predicted_char}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    # Show the processed frame with detection and OCR result
    cv2.imshow("License Plate Recognition", frame)

    # Exit condition on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close the OpenCV windows
cap.release()
cv2.destroyAllWindows()

In [ ]:
# Perform YOLO object detection
results = yolo_model(frame)  # Perform object detection using YOLO

# Extract bounding boxes and confidence scores from results
pred = results[0].boxes  # Get the box predictions (x, y, width, height, confidence)

# Loop through the detections
for box in pred:
    confidence = box.conf.item()  # Get the confidence score
    if confidence > 0.5:  # Only consider detections with confidence > 50%
        x1, y1, x2, y2 = map(int, box.xywh[0])  # Extract bounding box coordinates
        license_plate_image = frame[y1:y2, x1:x2]  # Extract the license plate image

        # Visualize the bounding box and confidence score
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw bounding box
        cv2.putText(frame, f'{confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)  # Display confidence

# Show the processed frame with bounding boxes
cv2.imshow("YOLO License Plate Detection", frame)


NameError: name 'frame' is not defined

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array

# Load YOLO model for license plate detection
yolo_model = YOLO("D:/anpr2/runs/detect/train_final.pt")  # Path to your YOLO model

# Load the trained character recognition OCR model
ocr_model = tf.keras.models.load_model("D:/anpr2/ocr_model_final.keras")  # Load the OCR model

# Start capturing video from webcam
cap = cv2.VideoCapture(0)  # Use 0 for webcam, or provide a video file path

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Perform YOLO object detection
    results = yolo_model(frame)

    # Extract bounding boxes and class predictions
    for result in results[0].boxes:  # Access the detected boxes from the first image
        confidence = result.conf[0].item()  # Confidence of the detected box
        if confidence > 0.5:  # Only consider detections with a confidence above 50%
            # Extract bounding box coordinates for license plate
            x1, y1, x2, y2 = map(int, result.xyxy[0])  # Extract coordinates from result
            license_plate_image = frame[y1:y2, x1:x2]  # Crop the license plate region

            # Preprocess the license plate image for OCR model (resize to 28x28 and convert to grayscale)
            license_plate_image = cv2.cvtColor(license_plate_image, cv2.COLOR_BGR2GRAY)
            license_plate_image = cv2.resize(license_plate_image, (28, 28))
            license_plate_image = img_to_array(license_plate_image)
            license_plate_image = np.expand_dims(license_plate_image, axis=0)
            license_plate_image = license_plate_image / 255.0  # Normalize the image

            # Use OCR model to predict characters
            ocr_result = ocr_model.predict(license_plate_image)
            
            # Decode the predicted character(s) from the OCR model output
            predicted_class = np.argmax(ocr_result, axis=-1)
            predicted_class = predicted_class.item()  # Convert to scalar if it's an array
            
            # Convert to character (A-Z for 0-25, 0-9 for 26-35)
            if predicted_class < 26:
                predicted_char = chr(predicted_class + 65)  # A-Z
            else:
                predicted_char = str(predicted_class - 26)  # 0-9

            # Display the recognized text on the image (removing the license plate text display for now)
            cv2.putText(frame, f"LP Detected: {predicted_char}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            
            # Draw bounding box around the detected license plate
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box

    # Show the processed frame with detection and OCR result
    cv2.imshow("License Plate Recognition", frame)

    # Exit condition on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close the OpenCV windows
cap.release()
cv2.destroyAllWindows()



0: 480x640 (no detections), 13.3ms
Speed: 2.5ms preprocess, 13.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 9.6ms
Speed: 2.3ms preprocess, 9.6ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 8.0ms
Speed: 2.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 8.5ms
Speed: 2.0ms preprocess, 8.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 9.8ms
Speed: 2.0ms preprocess, 9.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 10.1ms
Speed: 1.5ms preprocess, 10.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 10.6ms
Speed: 1.0ms preprocess, 10.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 12.7ms
Speed: 1.0ms preprocess, 12.7ms inference

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
import time

# Load YOLO model for license plate detection
yolo_model = YOLO("D:/anpr2/runs/detect/train_final.pt")  # Path to your YOLO model

# Load the trained character recognition OCR model
ocr_model = tf.keras.models.load_model("D:/anpr2/ocr_model_final.keras")  # Load the OCR model

# Start capturing video from webcam
cap = cv2.VideoCapture(0)  # Use 0 for webcam, or provide a video file path

# Frame processing control variables
frame_count = 0
fps = 0  # To calculate frames per second

while True:
    start_time = time.time()  # Start the timer for FPS calculation
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame. Exiting...")
        break

    frame_count += 1
    if frame_count % 5 != 0:  # Limit OCR predictions to every 5th frame
        continue

    # Perform YOLO object detection
    results = yolo_model(frame)
    
    # Debug: Print the number of detections
    print(f"YOLO detected {len(results[0].boxes)} objects")

    # Extract bounding boxes and class predictions
    detected_boxes = results[0].boxes
    if len(detected_boxes) == 0:
        print("No license plates detected")
        continue  # Skip if no license plates are detected

    # Extracting boxes from YOLO result and checking each detection
    for result in detected_boxes:
        confidence = result.conf[0].item()  # Confidence of the detected box
        if confidence > 0.5:  # Only consider detections with a confidence above 50%
            # Extract bounding box coordinates for license plate
            x1, y1, x2, y2 = map(int, result.xyxy[0])  # Extract coordinates from result
            license_plate_image = frame[y1:y2, x1:x2]  # Crop the license plate region

            # Debug: Check if the cropped image is empty
            if license_plate_image.size == 0:
                print("Empty license plate image detected, skipping OCR...")
                continue

            # Debug: Show cropped license plate image (for debugging purposes)
            cv2.imshow("Cropped License Plate", license_plate_image)

            # Preprocess the license plate image for OCR model (resize to 128x128 and convert to grayscale)
            license_plate_image = cv2.cvtColor(license_plate_image, cv2.COLOR_BGR2GRAY)
            license_plate_image = cv2.resize(license_plate_image, (128, 128))  # Increased resolution
            license_plate_image = img_to_array(license_plate_image)
            license_plate_image = np.expand_dims(license_plate_image, axis=0)
            license_plate_image = license_plate_image / 255.0  # Normalize the image

            # Debugging: Check the shape of the license plate image
            print(f"License plate image shape: {license_plate_image.shape}")

            # Use OCR model to predict characters (Wrapped in try-except block for safety)
            try:
                ocr_result = ocr_model.predict(license_plate_image)
                
                # Debugging: Check OCR result
                print(f"OCR result: {ocr_result}")

                # Decode the predicted character(s) from the OCR model output
                predicted_class = np.argmax(ocr_result, axis=-1)
                predicted_class = predicted_class.item()  # Convert to scalar if it's an array
                
                # Convert to character (A-Z for 0-25, 0-9 for 26-35)
                if predicted_class < 26:
                    predicted_char = chr(predicted_class + 65)  # A-Z
                else:
                    predicted_char = str(predicted_class - 26)  # 0-9

                # Display the recognized text on the image (removing the license plate text display for now)
                cv2.putText(frame, f"LP Detected: {predicted_char}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                
            except Exception as e:
                print(f"OCR prediction error: {e}")
                continue
            
            # Draw bounding box around the detected license plate
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box

    # Show the processed frame with detection and OCR result
    cv2.imshow("License Plate Recognition", frame)

    # Calculate FPS
    fps = 1 / (time.time() - start_time)
    print(f"FPS: {fps:.2f}")

    # Exit condition on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close the OpenCV windows
cap.release()
cv2.destroyAllWindows()



0: 480x640 (no detections), 10.1ms
Speed: 2.0ms preprocess, 10.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
YOLO detected 0 objects
No license plates detected

0: 480x640 (no detections), 10.6ms
Speed: 2.0ms preprocess, 10.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
YOLO detected 0 objects
No license plates detected

0: 480x640 (no detections), 7.8ms
Speed: 2.0ms preprocess, 7.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
YOLO detected 0 objects
No license plates detected

0: 480x640 (no detections), 10.4ms
Speed: 2.0ms preprocess, 10.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
YOLO detected 0 objects
No license plates detected

0: 480x640 (no detections), 9.8ms
Speed: 1.5ms preprocess, 9.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
YOLO detected 0 objects
No license plates detected

0: 480x640 (no detections), 10.8ms
Speed: 1.0ms preprocess, 10.8ms inference, 1.0ms

KeyboardInterrupt: 

: 

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
import os
import time
import csv

# Create a directory to save the cropped license plates if it doesn't exist
save_folder = "D:/anpr2/cropped_plates/"
os.makedirs(save_folder, exist_ok=True)

# Set up CSV file for saving detected license plate information
csv_file = "D:/anpr2/license_plate_data.csv"
csv_header = ['Frame', 'Plate Number', 'X1', 'Y1', 'X2', 'Y2', 'Saved Image Path']

# Initialize the CSV file (if it doesn't exist already)
if not os.path.exists(csv_file):
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(csv_header)

# Load YOLO model for license plate detection
yolo_model = YOLO("D:/anpr2/runs/detect/train_final.pt")  # Path to your YOLO model

# Load the trained character recognition OCR model
ocr_model = tf.keras.models.load_model("D:/anpr2/ocr_model_final.keras")  # Load the OCR model

# Start capturing video from webcam
cap = cv2.VideoCapture(0)  # Use 0 for webcam, or provide a video file path

# Frame processing control variables
frame_count = 0
fps = 0  # To calculate frames per second

while True:
    start_time = time.time()  # Start the timer for FPS calculation
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame. Exiting...")
        break

    frame_count += 1

    # Process every 5th frame to reduce OCR load
    if frame_count % 5 != 0:  
        continue

    # Perform YOLO object detection
    results = yolo_model(frame)
    
    print(f"YOLO detected {len(results[0].boxes)} objects")

    # Extract bounding boxes and class predictions
    detected_boxes = results[0].boxes
    if len(detected_boxes) == 0:
        print("No license plates detected")
        continue  # Skip if no license plates are detected

    # Extracting boxes from YOLO result and checking each detection
    for result in detected_boxes:
        confidence = result.conf[0].item()  # Confidence of the detected box
        if confidence > 0.5:  # Only consider detections with a confidence above 50%
            # Extract bounding box coordinates for license plate
            x1, y1, x2, y2 = map(int, result.xyxy[0])  # Extract coordinates from result
            license_plate_image = frame[y1:y2, x1:x2]  # Crop the license plate region

            # Debug: Check if the cropped image is empty
            if license_plate_image.size == 0:
                print("Empty license plate image detected, skipping OCR...")
                continue

            # Preprocess the license plate image for OCR model (resize to 40x40 or another suitable size)
            license_plate_image = cv2.cvtColor(license_plate_image, cv2.COLOR_BGR2GRAY)
            license_plate_image = cv2.resize(license_plate_image, (40, 40))  # Resize to 40x40 (example)
            license_plate_image = img_to_array(license_plate_image)
            license_plate_image = np.expand_dims(license_plate_image, axis=0)  # Expand to batch dimension
            license_plate_image = license_plate_image / 255.0  # Normalize the image

            # Flatten the image to match the model's input shape (1600,)
            license_plate_image = license_plate_image.flatten().reshape(1, -1)  # Flatten to a 1D vector
            
            # Debugging: Check the shape of the license plate image
            print(f"Flattened license plate image shape: {license_plate_image.shape}")

            # Use OCR model to predict characters (Wrapped in try-except block for safety)
            try:
                ocr_result = ocr_model.predict(license_plate_image)
                
                # Decode the predicted character(s) from the OCR model output
                predicted_class = np.argmax(ocr_result, axis=-1)
                predicted_class = predicted_class.item()  # Convert to scalar if it's an array
                
                # Convert to character (A-Z for 0-25, 0-9 for 26-35)
                if predicted_class < 26:
                    predicted_char = chr(predicted_class + 65)  # A-Z
                else:
                    predicted_char = str(predicted_class - 26)  # 0-9

                # Display the recognized text on the image
                cv2.putText(frame, f"LP Detected: {predicted_char}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                
            except Exception as e:
                print(f"OCR prediction error: {e}")
                continue

            # Save the cropped license plate image to a folder
            lp_filename = f"{save_folder}license_plate_{frame_count}_{x1}_{y1}.jpg"
            cv2.imwrite(lp_filename, frame[y1:y2, x1:x2])  # Save the cropped plate

            # Write the details of the detected license plate to CSV
            with open(csv_file, mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([frame_count, predicted_char, x1, y1, x2, y2, lp_filename])

            # Draw bounding box around the detected license plate
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box

    # Show the processed frame with detection and OCR result
    cv2.imshow("License Plate Recognition", frame)

    # Calculate FPS
    fps = 1 / (time.time() - start_time)
    print(f"FPS: {fps:.2f}")

    # Exit condition on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close the OpenCV windows
cap.release()
cv2.destroyAllWindows()


c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))



0: 480x640 (no detections), 90.8ms
Speed: 3.7ms preprocess, 90.8ms inference, 20.2ms postprocess per image at shape (1, 3, 480, 640)
YOLO detected 0 objects
No license plates detected

0: 480x640 (no detections), 9.9ms
Speed: 2.0ms preprocess, 9.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
YOLO detected 0 objects
No license plates detected

0: 480x640 (no detections), 10.6ms
Speed: 1.9ms preprocess, 10.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
YOLO detected 0 objects
No license plates detected

0: 480x640 (no detections), 17.7ms
Speed: 3.2ms preprocess, 17.7ms inference, 3.6ms postprocess per image at shape (1, 3, 480, 640)
YOLO detected 0 objects
No license plates detected

0: 480x640 (no detections), 14.1ms
Speed: 3.0ms preprocess, 14.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
YOLO detected 0 objects
No license plates detected

0: 480x640 (no detections), 15.8ms
Speed: 3.0ms preprocess, 15.8ms inference, 1.

KeyboardInterrupt: 

: 